In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

input_file = 'results.csv'

In [ ]:
df = pd.read_csv(input_file, comment='#')
markers = ['o', '+', 'x', '*', 'p', 'X', 'd', 'D', 'v', 's']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
class_names = sorted(set([re.sub('<.*>', "", x) for x in df.class_name.unique()]))
dataset_names = df.dataset.unique()
ncols = min(3, len(dataset_names))
nrows = -(-len(dataset_names) // ncols)
fig, axs = plt.subplots(constrained_layout=True, ncols=ncols, nrows=nrows, squeeze=False, figsize=(5.3 * ncols, 4.5 * nrows))

for dataset, ax in zip(dataset_names, axs.flat):
    ax.set_prop_cycle(marker=markers, color=colors)

    for class_name in class_names:
        d = df[(df.dataset == dataset) & (df.class_name.str.startswith(class_name))]
        ax.plot(d.bytes, d.query_ns, label=class_name, linestyle='None', markersize=5)

    max_bytes = df[(df.dataset == dataset)].bytes.max()
    try:
        from cpufeature import CPUFeature as cpu
        if cpu['cache_L1_size'] < max_bytes: ax.axvline(x=cpu['cache_L1_size'], lw=0.6, c='k', label='Cache sizes')
        if cpu['cache_L2_size'] < max_bytes: ax.axvline(x=cpu['cache_L2_size'], lw=0.6, c='k')
        if cpu['cache_L3_size'] < max_bytes: ax.axvline(x=cpu['cache_L3_size'], lw=0.6, c='k')
    except:
        print('To plot also the cache sizes, run: pip3 install cpufeature')

    ax.set_title(dataset)
    ax.set_xscale('log')
    ax.legend(fontsize='x-small')
    ax.grid(which='both', linestyle=':', lw=0.5)
    ax.set_ylabel('Query time (ns)')
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
    if max_bytes < 1e6:
        ax.set_xlabel('Index size (KB)')
        ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y / 1e3)))
    else:
        ax.set_xlabel('Index size (MB)')
        ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y / 1e6)))

plt.savefig('plot.pdf')
plt.show()